# Using Foreign Tables
### Global Summit 2023 - Demos and Drinks

With 2023.1, we're releasing Foreign Tables as an experimental feature of InterSystems IRIS and IRIS for Health. Foreign Tables is a new capability of IRIS SQL that enables users to include data managed elsewhere, in a file or remote database, in regular IRIS SQL queries. Use cases include scenarios in which the ownership, size and rate of change of the remote data make it impractical to load the data into IRIS. Unlike its predecessor Linked Tables that's often used in Interoperability use cases, Foreign Tables have no limitations on how they can be combined with other tables or the syntax that can be used to query them. 

## A Motivating Example

In this example, we're going to reconcile the data architecture for our brand new beachfront resort. Recently, management decided to go with InterSystems IRIS Data Platform for all our data needs -- a very wise decision indeed! Unfortunately, right after acquiring the resort from the previous owner we found out they stored **all** of their data in CSV files. To make matters worse, management originally decided to go with MySQL for the resort's booking system. Most recently, we've used InterSystems IRIS to store all our guest loyalty program data. So we've got a CSV file storing all our room information, a MySQL system storing our reservations, and IRIS storing our loyalty program data -- and it's up to us to bring it all together. 

Thankfully, management's decision to go with InterSystems is about to pay off as we can easily unite all this data directly in IRIS with the help of Foreign Tables. All we need to do is create two foreign tables (one to access our MySQL data, one to access our CSV data) and then get to querying just as if we were querying native data.

First, let's set up our Python environment to connect to our IRIS instance where we want to create our foreign tables:

In [ ]:
import iris, os
from tabulate import tabulate

# replace this with your connection info as needed
hostname = os.getenv("ISC_FT_HOSTNAME")
port = os.getenv("ISC_FT_PORT")
namespace = os.getenv("ISC_FT_NAMESPACE")
user = os.getenv("ISC_FT_USER")
pwd = os.getenv("ISC_FT_PASSWORD")

connection = iris.connect(str(hostname)+":"+str(port)+"/"+str(namespace), user, pwd)

import warnings
warnings.simplefilter("ignore")

def query(sql):
    try:
        c1 = connection.cursor()
        c1.execute(sql)
        print("Executed successully")
        if c1 is None:
            return
        resultSet = c1.fetchall()
        c1.close()
    except Exception as e:
        print("Query failed: " + str(e))
        return None
    return resultSet

def ddl(sql):
    try:
        c1 = connection.cursor()
        c1.execute(sql)
        print("Executed successully")
        c1.close()
    except Exception as e:
        print("Query failed: " + str(e))
    return

Now that we have our connection, let's make sure we can query our local IRIS table Resort.Guests. For most of our examples, we're only going to print the first 10 rows to avoid cluttering the notebook:

In [ ]:
rs = query("SELECT * FROM Resort.Guests")

print(tabulate(rs[0:10], headers = ['guest_id', 'first_name', 'last_name', 'rewards_level']))

While we're at it, let's also investigate the CSV file directly in Python by trying to find how many Double Queen rooms our resort has (and in the process perhaps feel some pity for the team at the old resort):

In [ ]:
import csv

with open('/opt/demodata/rooms.csv', newline='') as room_file:
    room_reader = csv.reader(room_file, delimiter=',')
    num_rooms = 0
    for row in room_reader:
        if row[1] == 'Double Queen':
            num_rooms += 1
            
print(f'There are {num_rooms} double queen rooms in the resort')

That works well enough for such a simple use case, but can we do better? Of course we can!

## Step 1 
### Creating a Foreign Server

So we can access our IRIS instance and see the data in our Resort.Guests table, but what about the data in our MySQL table and CSV file? First we have to define the Foreign Server representing either the external database server or a directory on the local filesystem.

In [ ]:
ddl("CREATE SERVER Resort.CSVServer FOREIGN DATA WRAPPER CSV HOST '/opt/irisbuild/data/'")

ddl("CREATE SERVER Resort.MySQLServer FOREIGN DATA WRAPPER JDBC CONNECTION 'mysql' NODELIMITEDIDS")

The first command creates the server in the Resort schema and ties it to the specified directory. It also tells IRIS to use the CSV Foreign Data Wrapper (FDW) for all tables under this server. You can think of a Foreign Data Wrapper as the adapter logic that InterSystems IRIS uses to interact with a specific type of data source. The second command creates the server in the same schema and ties it to the JDBC SQL Gateway connection named "mysql", and specifies a different FDW. We also include the NODELIMITEDIDS keyword so that IRIS won't wrap our identifier names in quotation marks when it sends queries to the source MySQL instance.


## Step 2
### Creating Foreign Tables

Now we'll create an actual Foreign Table, projecting a table from our MySQL database and CSV file:

In [ ]:
ddl("CREATE FOREIGN TABLE Resort.Rooms ( "+
               "room_no INT, room_type VARCHAR(50), description VARCHAR(500)"+
               ") SERVER Resort.CSVServer FILE 'rooms.csv' VALUES (room_no, room_type, description)")

ddl("CREATE FOREIGN TABLE Resort.Reservations "+
               "SERVER Resort.MySQLServer TABLE 'activereservations'")

The first command creates the table in the Resort schema on IRIS, tying it to the rooms.csv file in the directory specified for the Foreign Server. The second command creates the table in the same schema on IRIS, tying it to the activereservations table in the external MySQL instance. Notice how the first command specifies the column list explicitly while the second one doesn't: because JDBC query results (such as SELECT * FROM activereservations) come with structural metadata, there is no need to specify any column names and datatypes if you'd like to project the remote table as-is. We also use the VALUES clause to specify what the names of the columns are in the external source.

## Step 3
### Query!

Let's make sure that we can in fact acces both our foreign tables.  As an example, let's see if we can improve on the mess of Python we needed before for our CSV file and fire off one simple SQL query to see how many double queen rooms our resort has:

In [ ]:
rs = query("SELECT COUNT(*) FROM Resort.Rooms WHERE room_type = 'Double Queen'")

print(tabulate(rs, headers = ['Count']))

Much better! Let's query the MySQL instance as well, grabbing all the reservations with a rate between \\$400 and \\$500:

In [ ]:
rs = query("SELECT * FROM Resort.Reservations WHERE rate BETWEEN 400 AND 500")

print(tabulate(rs[0:10], headers = ['res_no','room_no','guest_id','rate','check_in_date','check_out_date','check_in_time','check_out_time']))

In the above query, since we're querying an external database IRIS will even send our condition to the DBMS to be processed remotely! We can see this from the [query plan](https://docs.intersystems.com/iris20231/csp/docbook/DocBook.UI.Page.cls?KEY=GSOE_interpret#GSOE_interpret_showplan)

    <plans>  
        <plan>
            <sql>
            SELECT * FROM Resort.Reservations WHERE rate BETWEEN 400 AND 500 /*#OPTIONS {""DynamicSQL"":1} */
            </sql>
            <cost value=""720400""/>
            Read foreign table Resort.Reservations, sending query:
                SELECT res_no,room_no,guest_id,rate,check_in_date,check_out_date,check_in_time,check_out_time   FROM activereservations T1 WHERE (T1.rate >= ((400))) AND (T1.rate <= ((500)))
            and looping on pseudo-ID.
            For each row:
                Output the row.
        </plan>
    </plans>

What if we wanted to include multiple tables in our query? We can do that! For example, let's grab some reservation information about Platinum rewards guests staying at the resort:

In [ ]:
rs = query("SELECT res_no, room_no, rate FROM Resort.Reservations res WHERE res.guest_id IN (SELECT guest_id FROM Resort.Guests WHERE rewards_level = 'Platinum')")

print(tabulate(rs[0:10], headers = ['res_no','room_no','rate']))

## Joins

Sure, it's useful to access the information in our CSV file and our MySQL table from IRIS but in order to *truly* leverage our data we need to bring it all together. Thankfully, foreign tables allow us to perform *hetereogeneneous joins*, or in other words, joins between tables from different data sources. This means we're **not** limited to joins from the same external server, and not even to data sources of the same type! To see this at work, let's try to get some information about all Platinum level guests that are staying in rooms for a rate of $500 or more:

In [ ]:
rs = query("SELECT g.guest_id, g.first_name, g.last_name, res.rate "+
           "FROM Resort.Guests g JOIN Resort.Reservations res ON g.guest_id = res.guest_id "+
           "WHERE res.rate > 500")

print(tabulate(rs[0:10], headers=['guest_id','first_name','last_name','rate']))

And just like that, we've got results joined from our local IRIS table and our remote MySQL table! But what if we wanted to join results from our CSV-based table? Not a problem! Let's grab the average rate being paid for each room type:

In [ ]:
rs = query("SELECT room.room_type, AVG(res.rate) FROM Resort.Reservations res "+
           "JOIN Resort.Rooms room ON res.room_no = room.room_no GROUP BY room.room_type")

print(tabulate(rs, headers=['room_type', 'avg_rate']))

We've just aggregated data from a MySQL instance, filtered by joining with a table based in a CSV file, while accessing the results directly in IRIS and leveraging InterSystems IRIS' SQL engine. Though we didn't in this example, we could've even used some IRIS-SQL-specific capabilities like %-functions. No special syntax required, no hoops to jump through, just InterSystems IRIS and your data -- wherever it may live.

## Wrapping Up

This quick demo is just a taste of what our new Foreign Tables feature can do for you and your data. Feel free to play around with this Jupyter Notebook and see what types of queries you can put together, or download the IRIS Community Edition locally and create your own Foreign Tables to experiment with. As mentioned above, Foreign Tables were released in 2023.1 as an experimental feature and we look forward to graduating it to a production-ready feature. As such, we're always looking to iterate and improve so please reach out with feedback, issues, or questions. We'd also love to hear from users about their individual use cases to help us improve the feature moving forward!

If you have any questions or comments feel free to send them to me at mgolden@intersystems.com (or better yet if you see me at the summit, stop me and ask away)!